## GEN

In [19]:
import os
import re
import pandas as pd
from datetime import datetime, timedelta

# Set the folder path where your Excel files are located
folder_path = r"C:\Users\34673\Documents\Universidad\Master\Florida Tech\Neural Networks\Project - Solar\Datass"  # Replace with your actual folder path

# Dictionary to store period information for each file
file_periods = {}
all_periods = []

# Regular expression to extract the period from cell A1
period_pattern = r"Period: (\d{2}/\d{2}/\d{4}) to (\d{2}/\d{2}/\d{4})"

# Function to convert date string to datetime object
def convert_to_date(date_str):
    return datetime.strptime(date_str, "%d/%m/%Y")

# Function to convert datetime to string in the format we need
def date_to_str(date_obj):
    return date_obj.strftime("%d%m%Y")

# Step 1: Scan the folder and extract period information
for filename in os.listdir(folder_path):
    if filename.endswith(('.xls', '.xlsx')):
        file_path = os.path.join(folder_path, filename)
        
        try:
            # Try to read using pandas with default engine
            df = pd.read_excel(file_path, nrows=1, usecols=[0], engine=None)
            period_text = df.iloc[0, 0]
            period_text = df.columns[0]
                        
            # Extract the period dates using regex
            match = re.search(period_pattern, period_text)
            if match:
                start_date = match.group(1)
                end_date = match.group(2)
                
                # Convert to datetime objects
                start_dt = convert_to_date(start_date)
                end_dt = convert_to_date(end_date)
                
                # Create a period identifier
                period_id = f"{str(start_dt)[:11]}_to_{str(end_dt)[:11]}"
                
                # Store the period information
                if period_id in file_periods:
                    file_periods[period_id].append(filename)
                else:
                    file_periods[period_id] = [filename]
                
                all_periods.append((start_dt, end_dt))
                print(f"Processed {filename}: {start_date} to {end_date}")
        except Exception as e:
            print(f"Error processing {filename}: {e}")

# Step 2: Find duplicate periods
duplicates = {period: files for period, files in file_periods.items() if len(files) > 1}

# Step 3: Find missing periods
# Sort periods by start date
all_periods.sort()

if all_periods:
    # Get the earliest and latest dates
    earliest_date = all_periods[0][0]
    latest_date = all_periods[-1][1]
    
    # Determine the period length (assuming all periods have the same length)
    # Using the first period as reference
    first_period_length = (all_periods[0][1] - all_periods[0][0]).days + 1
    
    # Generate all expected periods
    expected_periods = []
    current_date = earliest_date
    
    while current_date <= latest_date:
        end_date = current_date + timedelta(days=first_period_length - 1)
        expected_periods.append((current_date, end_date))
        current_date = end_date + timedelta(days=1)
    
    # Find missing periods
    missing_periods = []
    for period in expected_periods:
        if period not in all_periods:
            missing_periods.append(period)

# Step 4: Rename the files
for period_id, filenames in file_periods.items():
    for i, filename in enumerate(filenames):
        old_path = os.path.join(folder_path, filename)
        new_filename = f"{period_id}.xls"
        
        # Add a suffix if there are duplicates
        if len(filenames) > 1:
            new_filename = f"{period_id}_copy{i+1}.xls"
        
        new_path = os.path.join(folder_path, new_filename)
        
        try:
            os.rename(old_path, new_path)
            print(f"Renamed: {filename} -> {new_filename}")
        except Exception as e:
            print(f"Error renaming {filename}: {e}")

# Print summary
print("\nSummary:")
print(f"Total files processed: {len(file_periods)}")
print(f"Earliest period: {earliest_date.strftime('%d/%m/%Y')} to {all_periods[0][1].strftime('%d/%m/%Y')}")
print(f"Latest period: {all_periods[-1][0].strftime('%d/%m/%Y')} to {latest_date.strftime('%d/%m/%Y')}")

if duplicates:
    print("\nDuplicate periods found:")
    for period, files in duplicates.items():
        print(f"  {period}: {len(files)} files")

if missing_periods:
    print("\nMissing periods:")
    for start, end in missing_periods:
        print(f"  {start.strftime('%d/%m/%Y')} to {end.strftime('%d/%m/%Y')}")

Processed 2019-01-01 _to_2019-01-31 .xls: 01/01/2019 to 31/01/2019
Processed 2019-02-01 _to_2019-02-28 _copy1.xls: 01/02/2019 to 28/02/2019
Processed 2019-03-01 _to_2019-03-31 _copy1.xls: 01/03/2019 to 31/03/2019
Processed 2019-04-01 _to_2019-04-30 _copy1.xls: 01/04/2019 to 30/04/2019
Processed 2019-05-01 _to_2019-05-31 _copy1.xls: 01/05/2019 to 31/05/2019
Processed 2019-06-01 _to_2019-06-30 _copy1.xls: 01/06/2019 to 30/06/2019
Processed 2019-07-01 _to_2019-07-31 _copy1.xls: 01/07/2019 to 31/07/2019
Processed 2019-08-01 _to_2019-08-31 _copy1.xls: 01/08/2019 to 31/08/2019
Processed 2019-09-01 _to_2019-09-30 _copy1.xls: 01/09/2019 to 30/09/2019
Processed 2019-10-01 _to_2019-10-31 _copy1.xls: 01/10/2019 to 31/10/2019
Processed 2019-11-01 _to_2019-11-30 _copy1.xls: 01/11/2019 to 30/11/2019
Processed 2019-12-01 _to_2019-12-31 .xls: 01/12/2019 to 31/12/2019
Processed 2020-01-01 _to_2020-01-31 _copy1.xls: 01/01/2020 to 31/01/2020
Processed 2020-02-01 _to_2020-02-29 _copy1.xls: 01/02/2020 to 2

In [21]:
import os
import pandas as pd
import glob

# Set the folder path where your .xls files are located
  # Replace with your actual folder path

# Set the output file path
output_file = os.path.join(folder_path, "combined_data.xlsx")

# Get a list of all .xls files in the folder
excel_files = glob.glob(os.path.join(folder_path, "*.xls"))

# Sort the files by name to maintain order
excel_files.sort()

# Create an empty list to store DataFrames
all_data = []

# Process each Excel file
for i, file_path in enumerate(excel_files):
    try:
        # Read the Excel file, skipping the first 3 rows
        # The 4th row (index 3) contains column headers
        if i == 0:
            # For the first file, read the headers (row 4) and use them
            df = pd.read_excel(file_path, header=3)
            print(f"Headers from first file: {', '.join(df.columns.tolist())}")
        else:
            # For subsequent files, skip headers and use the ones from the first file
            df = pd.read_excel(file_path, header=3)
        
        # Add the DataFrame to our list
        all_data.append(df)
        print(f"Processed: {os.path.basename(file_path)} - {len(df)} rows")
        
    except Exception as e:
        print(f"Error processing {file_path}: {e}")

# Check if we have any data
if not all_data:
    print("No data was processed. Check your file path and file formats.")
else:
    # Combine all DataFrames into one
    combined_df = pd.concat(all_data, ignore_index=True)
    
    # Save the combined data to a new Excel file
    combined_df.to_excel(output_file, index=False)
    
    print(f"\nCombined data saved to: {output_file}")
    print(f"Total rows: {len(combined_df)}")

Headers from first file: DateTime, Most recent forecast [MW], Day-Ahead forecast [MW], Week-Ahead forecast [MW], Real-time Upscaled Measurement [MW], Corrected Upscaled Measurement [MW], Monitored Capacity [MWp], Day-Ahead forecast (11h00) [MW]
Processed: 2019-01-01 _to_2019-01-31 .xls - 2976 rows
Processed: 2019-02-01 _to_2019-02-28 .xls - 2688 rows
Processed: 2019-03-01 _to_2019-03-31 .xls - 2972 rows
Processed: 2019-04-01 _to_2019-04-30 .xls - 2880 rows
Processed: 2019-05-01 _to_2019-05-31 .xls - 2976 rows
Processed: 2019-06-01 _to_2019-06-30 .xls - 2880 rows
Processed: 2019-07-01 _to_2019-07-31 .xls - 2976 rows
Processed: 2019-08-01 _to_2019-08-31 .xls - 2976 rows
Processed: 2019-09-01 _to_2019-09-30 .xls - 2880 rows
Processed: 2019-10-01 _to_2019-10-31 .xls - 2980 rows
Processed: 2019-11-01 _to_2019-11-30 .xls - 2880 rows
Processed: 2019-12-01 _to_2019-12-31 .xls - 2976 rows
Processed: 2020-01-01 _to_2020-01-31 .xls - 2976 rows
Processed: 2020-02-01 _to_2020-02-29 .xls - 2784 rows

## METEO

In [3]:
import openmeteo_requests
import requests_cache
from retry_requests import retry
import pandas as pd
from datetime import datetime

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Specify coordinates for the location you're interested in
latitude = 50.6403  # Example: Berlin
longitude = 4.6667

# Define the API request parameters
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": latitude,
    "longitude": longitude,
    "start_date": "2019-01-01",  # Adjust to your desired date range
    "end_date": "2025-03-31",
    "hourly": ["temperature_2m", "cloud_cover", "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high", "precipitation_probability", "weather_code"],
    "timezone": "auto"
}

# Make the API request
responses = openmeteo.weather_api(url, params=params)
response = responses[0]

# Process hourly data
hourly = response.Hourly()
hourly_cloud_cover = hourly.Variables(0).ValuesAsNumpy()
hourly_cloud_cover_low = hourly.Variables(1).ValuesAsNumpy()
hourly_cloud_cover_mid = hourly.Variables(2).ValuesAsNumpy()
hourly_cloud_cover_high = hourly.Variables(3).ValuesAsNumpy()
hourly_temperature_2m = hourly.Variables(4).ValuesAsNumpy()
hourly_precipitation_probability = hourly.Variables(5).ValuesAsNumpy()
hourly_weather_code = hourly.Variables(6).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["cloud_cover_low"] = hourly_cloud_cover_low
hourly_data["cloud_cover_mid"] = hourly_cloud_cover_mid
hourly_data["cloud_cover_high"] = hourly_cloud_cover_high
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["precipitation_probability"] = hourly_precipitation_probability
hourly_data["weather_code"] = hourly_weather_code

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)


# Save to CSV (optional)
hourly_dataframe.to_csv("weather_data__in_Belgium.csv", index=False)

                           date  cloud_cover  cloud_cover_low  \
0     2018-12-31 22:00:00+00:00        6.324             97.0   
1     2018-12-31 23:00:00+00:00        6.024             99.0   
2     2019-01-01 00:00:00+00:00        6.174             97.0   
3     2019-01-01 01:00:00+00:00        6.324            100.0   
4     2019-01-01 02:00:00+00:00        6.274            100.0   
...                         ...          ...              ...   
54763 2025-03-31 17:00:00+00:00       10.774              4.0   
54764 2025-03-31 18:00:00+00:00        8.824              7.0   
54765 2025-03-31 19:00:00+00:00        7.524              3.0   
54766 2025-03-31 20:00:00+00:00        6.824              0.0   
54767 2025-03-31 21:00:00+00:00        5.974              1.0   

       cloud_cover_mid  cloud_cover_high  temperature_2m  \
0                 96.0               0.0            30.0   
1                 93.0               0.0            80.0   
2                100.0               0.

In [5]:
df_met = pd.read_csv('weather_data__in_Belgium.csv', delimiter=',')
display(df_met)

,date,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,temperature_2m,precipitation_probability,weather_code
0,2018-12-31 22:00:00+00:00,6.324,97.0,96.0,0.0,30.0,NaN,3.0
1,2018-12-31 23:00:00+00:00,6.024,99.0,93.0,0.0,80.0,NaN,3.0
2,2019-01-01 00:00:00+00:00,6.174,97.0,100.0,0.0,88.0,NaN,3.0
3,2019-01-01 01:00:00+00:00,6.324,100.0,100.0,0.0,87.0,NaN,3.0
4,2019-01-01 02:00:00+00:00,6.274,100.0,100.0,0.0,80.0,NaN,3.0
...,...,...,...,...,...,...,...,...
54763,2025-03-31 17:00:00+00:00,10.774,4.0,4.0,0.0,0.0,NaN,0.0
54764,2025-03-31 18:00:00+00:00,8.824,7.0,7.0,0.0,0.0,NaN,0.0
54765,2025-03-31 19:00:00+00:00,7.524,3.0,3.0,0.0,0.0,NaN,0.0
54766,2025-03-31 20:00:00+00:00,6.824,0.0,0.0,0.0,0.0,NaN,0.0


## Sun

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pytz
from pysolar.solar import get_altitude, get_azimuth

def get_sun_angles_for_day(latitude, longitude, date=None, local_timezone=None, interval_minutes=15):
    """
    Calculate sun angles for an entire day at specified intervals using Pysolar.
    
    Parameters:
    -----------
    latitude : float
        Latitude in degrees (positive for North, negative for South)
    longitude : float
        Longitude in degrees (positive for East, negative for West)
    date : datetime, optional
        Base date for calculations (time component will be ignored)
    local_timezone : str, optional
        Timezone name (e.g., 'America/New_York')
    interval_minutes : int, optional
        Interval between calculations in minutes. Default is 15.
        
    Returns:
    --------
    times : list
        List of datetime objects
    altitudes : numpy array
        Sun altitude angles in degrees
    azimuths : numpy array
        Sun azimuth angles in degrees
    """
    # Set up timezone
    if local_timezone:
        tz = pytz.timezone(local_timezone)
    else:
        tz = pytz.UTC
    
    # If no date provided, use today
    if date is None:
        date = datetime.now(tz)
    elif not date.tzinfo:
        date = tz.localize(date)
    
    # Strip time component to get just the date
    date = date.replace(hour=0, minute=0, second=0, microsecond=0)
    
    # Calculate number of intervals in a day
    intervals_per_day = 24 * 60 // interval_minutes
    
    # Create time points for the entire day
    times = [date + timedelta(minutes=i * interval_minutes) for i in range(intervals_per_day)]
    
    # Convert times to UTC for Pysolar calculations
    utc_times = [t.astimezone(pytz.UTC) for t in times]
    
    # Calculate sun position for each time point
    altitudes = []
    azimuths = []
    
    for t in utc_times:
        # Pysolar expects UTC time
        alt = get_altitude(latitude, longitude, t)
        az = get_azimuth(latitude, longitude, t)
        altitudes.append(alt)
        azimuths.append(az)
    
    return times, np.array(altitudes), np.array(azimuths)

def main():
    # Example location: New York City
    latitude = 50.6403
    longitude = 4.6667
    timezone_str = 'Europe/Madrid'  # Use None for UTC

    tempos, altituds, azmuths = [], [], []
    
    # Get today's date (or specify a particular date)
    date_1 = datetime(2019, 1, 1)
    date_2 = datetime(2025, 3, 31)

    d_day = date_1
    while d_day <= date_2:
        # Calculate sun angles for the day
        times, altitudes, azimuths = get_sun_angles_for_day(
            latitude, longitude, date=d_day, local_timezone=timezone_str)
        d_day += timedelta(days=1)

        tempos.extend(times)
        altituds.extend(altitudes)
        azmuths.extend(azimuths)
    
    # Create a pandas DataFrame with the results
    results = pd.DataFrame({
        'Time': tempos,
        'Altitude (degrees)': altituds,
        'Azimuths': azmuths
    })
    
    # Return the vectors for altitude and azimuth
    return results

if __name__ == "__main__":
    data = main()

In [12]:
data.to_csv('Sun_angles.csv')

In [14]:
x = data
x['Altitude (degrees)'] = [0 if alt < 0 else alt for alt in data['Altitude (degrees)']]
x.head(-20)
x.to_csv('Solar_angle.csv')